In [1]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D, Dropout, Embedding, GlobalAveragePooling1D, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

import os
import re
import shutil
import string
import numpy as np
import matplotlib.pyplot as plt

In [2]:
print('tensorflow version is {}'.format(tf.__version__))

tensorflow version is 2.8.0


In [3]:
file='stack_overflow_16k'
url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

In [4]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir=file)

In [5]:
# # 뭔가 이상함, cache_subdir=file로 한 후 '' 해야 directory 구조가 원하는 대로 됨
dataset = tf.keras.utils.get_file(file, origin=url, untar=True, cache_dir='.' , cache_subdir='')

In [6]:
os.listdir(dataset)

['.DS_Store', 'test', 'stack_overflow_16k.tar.gz', 'README.md', 'train']

In [7]:
train_dir = os.path.join(dataset, 'train')
train_dir_list = os.listdir(train_dir)

In [8]:
train_dir_list

['.DS_Store', 'python', 'java', 'csharp', 'javascript']

In [9]:
train_dir_list = [i for i in train_dir_list if '.' != i[0]]

In [10]:
train_dir_list

['python', 'java', 'csharp', 'javascript']

In [11]:
indexs = np.random.randint(0, 2000, len(train_dir_list))

for idx, kind in enumerate(train_dir_list):
  file = os.listdir(os.path.join(train_dir, kind))[indexs[idx]]
  file = os.path.join(os.path.join(train_dir, kind), file)
  print(file)
  with open(file) as f:
    print(f.read())

./stack_overflow_16k/train/python/33.txt
"return the smallest element value in a list of lists given a list of lists like this :..[[""filea"",7],[""fileb"",4],[""filec"",17],[""filed"",15]]...how would you return the first element associated to the smallest value ? in this instance ""fileb"" since it has the smallest value (4). i'm guessing the shortest way would be using list comprehension ."

./stack_overflow_16k/train/java/302.txt
"basic blank program questions public class a { //some fields (may not only primitive types) and methods here}..   public class b {// some fields (may not only primitive types) and methods here, may also have a class a reference }...questions are followed :..    public class test{ ..public static void main(string[] args){ ..a a = new a();//1. it will allocate memory for one object of a and a is the reference points to that space? ..arraylist&lt;b&gt; blist = new arraylist&lt;b&gt;(10);//2. it will allocate memory for 10 objects of b?..arraylist&lt;b&gt; bl

In [12]:
batch_size = 32
seed = 42

raw_train = tf.keras.utils.text_dataset_from_directory(
    'stack_overflow_16k/train',
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Metal device set to: Apple M1


2022-06-09 17:01:21.685858: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-09 17:01:21.686016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
